In [1]:
%%capture
!pip3 install penn_grader --upgrade
!pip3 install pyaml

In [1]:
from penngrader.backend import PennGraderBackend

# PennGrader: Teacher Backend

Follow this notebook's instructions to write your homework's test cases and see student's grades.
### Configuration
Edit the AWS Lambda endpoint info here (or skip this cell and copy a `config.yaml`):

In [5]:
%%writefile config.yaml
config_api_url: 'https://...amazonaws.com/default/HomeworkConfig'
config_api_key: '...'

Overwriting config.yaml


In [2]:
# Set the homework number accordingly
backend = PennGraderBackend('config.yaml', 1)

Success! Teacher backend initialized.

Homework ID: CISDEMO_Spring_2019_HW1


### Metadata
Edit the following metadata variables to your preference:

`TOTAL_SCORE`: Total number of points this homework is worth. **Note:** Make sure all test cases weigths add up to this number.

`DEADLINE`:... you guessed it, the homework deadline. (_Format:_ `'YYYY-MM-DD HH:MM A'`)

`MAX_DAILY_TEST_CASE_SUBMISSIONS`: Maximum number of daily submissions per test case per student.

In [3]:
TOTAL_SCORE = 20
DEADLINE = '2022-12-05 11:59 PM'
MAX_DAILY_TEST_CASE_SUBMISSIONS = 100

In [4]:
backend.update_metadata(DEADLINE, TOTAL_SCORE, MAX_DAILY_TEST_CASE_SUBMISSIONS)

Success! Metadata updated.

Total HW Points: 20
Deadline: 2022-12-05 23:59:00
Max daily submissions per test case: 100



### Test Cases
Define a test case function for each question.

A test case function takes in a single input containing the student's answer and returns a tuple `(student score:int, maximum score:int)`. See example below:


In [5]:
import numpy as np
import pandas as pd

def test_case_1(answer): 
    # [answer] in this test case is a function that adds 2 numbers
    
    # First initalize the max_score score of this test case. 
    max_score     = 5
    student_score = 0

    # Since answer is a function that takes two parameters 
    # answer(1,2) should return 3 if implemented correctly.
    if answer(1,2) == 3:
        student_score += 3
    if answer(2,2) == 4:
        student_score += 2
    
    # Returning a (student_score, max_score) tuple of ints
    return (student_score, max_score)

def test_case_2(answer): 
    max_score     = 10
    student_score = 0
    
    students_df = answer 
    
    students_df = students_df.sort_values('first_name', ascending = False)
    if students_df.iloc[0].first_name == 'Leonardo':
        student_score += 10
        
    return (student_score, max_score)

Run the following cell to update the test cases.

In [6]:
backend.update_test_cases(globals().items())

Importing packages  {'types', 'json', 'inspect', 'datetime', 'numpy', 'yaml', 'difflib', 'ast', 'urllib', 'base64', 'pandas', 'dill'}
Importing libraries  {('json', 'json'), ('numpy', 'np'), ('inspect', 'inspect'), ('dill', 'dill'), ('pandas', 'pd'), ('base64', 'base64'), ('ast', 'ast'), ('types', 'types'), ('urllib', 'urllib')}
Importing functions  set()
Adding case {} test_case_1
Adding case {} test_case_2
Success: Test cases updated successfully.
